# **Inference_yolo_OBB_to_cvat**

In [8]:
model_path = r"C:\Users\CorneAI\YOLOv11_OBB\models\yolo11n_obb_1-295_1to139.pt"
image_parent_path = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\images"

# すでにアノーテーションされているラベルは変えずに、その次の画像から推論を行う
# CVATの仕様上、2より12や134が優先されることに留意
start_id = 501 #まだinferenceされていない最初のID (xmlから確認)
end_id = 1000 #Inferenceしたい最後のID（xmlから確認）

# すでにアノーテーションされているxmlファイルのパス
orig_xml_path = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_1to139_cvat.xml"

# 推論を行ったxmlファイルのパス（すでに行われているアノーテーションは変えない）
output_xml_path = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_139to160_cvat.xml"
output_temp_label_dir = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\temp_labels"

In [9]:
import xml.etree.ElementTree as ET
import os

image_paths = []
try:
    # XMLファイルを解析
    tree = ET.parse(orig_xml_path)
    root = tree.getroot()

    # <image> タグをすべて検索
    for image_tag in root.findall('.//image'): # .//image でネストされたタグも検索
        image_id_str = image_tag.get('id')
        name = image_tag.get('name')

        if image_id_str is None:
            print(f"警告: 'id' 属性が見つからない画像タグがあります。")
            continue
        if name is None:
            print(f"警告: ID {image_id_str} の画像タグに 'name' 属性が見つかりません。")
            continue

        try:
            image_id = int(image_id_str)
            # IDが指定された範囲内にあるか確認
            if start_id <= image_id <= end_id:
                # ここではXML内のパスをそのままリストに追加します。
                # 必要に応じて、ベースディレクトリと結合して絶対パスにしてください。
                # 例: base_image_dir = os.path.dirname(orig_xml_path) # または適切な画像ディレクトリ
                #    full_path = os.path.join(base_image_dir, name)
                #    image_paths.append(full_path)
                image_paths.append(name)
        except ValueError:
            print(f"警告: ID '{image_id_str}' を整数に変換できません。スキップします。")
            continue

    print(f"XMLファイルから ID {start_id} から {end_id} の範囲で {len(image_paths)} 個の画像パスを読み込みました。")
    print("最初の5件:", image_paths[:5]) # 確認用
    print("最後の5件:", image_paths[-5:]) # 確認用


except FileNotFoundError:
    print(f"エラー: XMLファイルが見つかりません: {orig_xml_path}")
except ET.ParseError:
    print(f"エラー: XMLファイルの解析に失敗しました: {orig_xml_path}")
except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")

# image_paths リストが作成されました。
# このリストは、XMLファイルに記載されている画像のうち、指定されたID範囲内の画像パス（または名前）を含みます。


XMLファイルから ID 501 から 1000 の範囲で 500 個の画像パスを読み込みました。
最初の5件: ['14-20190604-77-111937_9391b3f869f19372da08665560d681a75a00a0808d6dfa7231bd7038216a968a_L.jpg', '14-20190604-77-111937_9391b3f869f19372da08665560d681a75a00a0808d6dfa7231bd7038216a968a_R.jpg', '14-20190613-77-151229_06cabbaf8542ce06f9ff051a4876424ee59010556d22ff251a66988ab08ac2e2_L.jpg', '14-20190613-77-151229_06cabbaf8542ce06f9ff051a4876424ee59010556d22ff251a66988ab08ac2e2_R.jpg', '14-20190613-77-151229_d212e16e23d9b4892b51272d659d75dd3b00f4da25246cc83626fb40b2322923_R.jpg']
最後の5件: ['160-20150729-3-111133_fb627ae6fa6f7e295620dff98889192e0dc82967f43c460c6e60330a3742a326_R.jpg', '160-20160727-4-112519_35047c79168e184347d1b79e12dbc6fdeb5c68889559c2f71c8a1ce3e082ad4b_L.jpg', '160-20160727-4-112519_35047c79168e184347d1b79e12dbc6fdeb5c68889559c2f71c8a1ce3e082ad4b_R.jpg', '160-20160727-4-112519_3ab35ca5a4f47fe49bb7ad4eac7aadef2952a3b64a3f8196083f98c3ba6a1434_L.jpg', '160-20160727-4-112519_3ab35ca5a4f47fe49bb7ad4eac7aadef2952a3b64a3f819

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import random # 色をランダムに生成する場合（今回は固定リストを使用）

# モデルのロード
model = YOLO(model_path)  # あなたのカスタムモデルのパスを指定

# クラス名を取得
class_names = model.names

# クラスごとの色を定義 (BGR形式)
# 十分な数の異なる色を用意します。
colors = [
    (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (192, 192, 192), (128, 128, 128), (128, 0, 0),
    (128, 128, 0), (0, 128, 0), (128, 0, 128), (0, 128, 128), (0, 0, 128),
    (255, 165, 0), (255, 192, 203), (75, 0, 130), (0, 100, 0), (139, 69, 19)
]
num_classes = len(class_names)
# 必要であれば、クラス数に応じてランダムな色を生成することもできます
# if num_classes > len(colors):
#     for _ in range(num_classes - len(colors)):
#         colors.append((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))
num_colors = len(colors)


# 推論の実行
for image_name in image_paths[0:5]:
    # 画像のフルパスを作成
    image_path = Path(image_parent_path) / image_name

    # 画像の読み込み (BGR形式)
    original_img = cv2.imread(str(image_path))
    if original_img is None:
        print(f"警告: 画像を読み込めませんでした: {image_path}")
        continue

    # 推論実行
    results = model(str(image_path))

    # 検出結果を描画 (BGR画像に対して行う)
    result_img = original_img.copy()
    for result in results:
        # OBBの取得
        for obb in result.obb:
            # 4点の座標
            pts = obb.xyxyxyxy.cpu().numpy().astype(np.int32).reshape(-1, 2)

            # 中心、軸、角度を計算
            center = np.mean(pts, axis=0).astype(int)
            # 軸の長さ計算の安定性を向上させる (隣接する点を使用)
            width = np.linalg.norm(pts[0] - pts[1])
            height = np.linalg.norm(pts[1] - pts[2])
            axes = (int(width / 2), int(height / 2))
            # 角度計算の安定性を向上させる (atan2を使用)
            vec = pts[1] - pts[0]
            angle = np.degrees(np.arctan2(vec[1], vec[0]))

            # クラスIDと色の取得
            cls_id = int(obb.cls.item())
            color = colors[cls_id % num_colors] # クラスIDに応じて色を選択

            # 楕円を描画 (クラスごとの色で)
            cv2.ellipse(result_img, tuple(center), axes, angle, 0, 360, color, 2)

            # クラス名と信頼度
            class_name = class_names[cls_id]
            conf = obb.conf.item()
            label = f"{class_name} {conf:.2f}"

            # ラベルの表示位置を計算 (バウンディングボックスの左上付近)
            # 最もy座標が小さい点（最も上にある点）を探す
            top_point_idx = np.argmin(pts[:, 1])
            text_x = pts[top_point_idx, 0]
            text_y = pts[top_point_idx, 1] - 10 # 点の少し上に表示

            # 画像の上端からはみ出さないように調整
            text_y = max(10, text_y)
            # 画像の左端からはみ出さないように調整 (必要であれば)
            # text_x = max(0, text_x)

            # ラベルの背景を描画（任意）
            (label_width, label_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            # cv2.rectangle(result_img, (text_x, text_y + baseline), (text_x + label_width, text_y - label_height), color, cv2.FILLED)

            # ラベルを描画 (クラスごとの色で、計算した位置に)
            cv2.putText(result_img, label, (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2) # 線の色と同じにする

    # 結果を表示 (表示のためにBGRからRGBに変換)
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.title(image_name) # 画像ファイル名を表示
    plt.axis('off')
    plt.show()

In [29]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt # Keep for potential visualization during debugging
import numpy as np
from pathlib import Path
import xml.etree.ElementTree as ET
import math # For atan2 and degrees conversion

# --- Configuration ---
# あなたのカスタムモデルのパスを指定
# model_path = r"C:\path\to\your\best.pt" # Example path, replace with yours
model_path = r"C:\Users\CorneAI\YOLOv11_OBB\models\yolo11n_obb_1-295_1to139.pt" # Replace with your actual model path
# すでにアノーテーションされているxmlファイルのパス
orig_xml_path = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_1to139_cvat.xml"
# 推論を行ったxmlファイルのパス（すでに行われているアノーテーションは変えない）
output_xml_path = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_1to160_cvat_updated.xml" # Output file (updated name)
# 画像が格納されている親フォルダのパス
image_parent_path = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\images" # Replace with your actual image folder path
# 推論を実行する画像のファイル名リスト



# モデルのロード
try:
    model = YOLO(model_path)
except Exception as e:
    print(f"Error loading YOLO model from {model_path}: {e}")
    exit()

# クラス名を取得
class_names = model.names
print(f"Model class names: {class_names}")

# XMLファイルを解析
try:
    tree = ET.parse(orig_xml_path)
    root = tree.getroot()
    print(f"Successfully parsed XML: {orig_xml_path}")
except ET.ParseError as e:
    print(f"Error parsing XML file {orig_xml_path}: {e}")
    exit()
except FileNotFoundError:
    print(f"Error: Original XML file not found at {orig_xml_path}")
    exit()

# 推論の実行とXMLへの追加
processed_images_count = 0
annotations_added_count = 0
visualized_count = 0 # 視覚化カウンター

# --- 視覚化設定 ---
visualize_results = False  # Trueにすると結果を画像で表示
visualize_limit = 100      # 視覚化する画像の最大枚数 (0以下または指定なしなら無制限)
# --- ここまで ---


for image_name in image_paths:

    # 視覚化モードで制限枚数に達したらループを抜ける
    if visualize_results and visualize_limit > 0 and visualized_count >= visualize_limit:
        print(f"\nVisualization limit ({visualize_limit} images) reached.")
        break

    # 画像のフルパスを作成
    image_path = Path(image_parent_path) / image_name
    print(f"\nProcessing image: {image_name}")

    # 画像の読み込み (BGR形式) - 主にサイズ取得のため
    original_img = cv2.imread(str(image_path))
    if original_img is None:
        print(f"  Warning: Could not read image: {image_path}. Skipping.")
        continue

    img_height, img_width, _ = original_img.shape

    # XML内で対応するimage要素を検索
    image_element = root.find(f".//image[@name='{image_name}']")

    if image_element is None:
        # XMLにimage要素が存在しない場合、新しいimage要素を作成 (ただし、視覚化モードでない場合のみ実際にXMLツリーに追加)
        print(f"  Warning: Image '{image_name}' not found in XML {orig_xml_path}.")
        if not visualize_results:
            print(f"        Creating a new <image> tag and adding it to the XML tree.")
            image_element = ET.SubElement(root, 'image')
            # 新しいIDを決定 (既存の最大ID+1)
            all_ids = [int(img.get('id', '-1')) for img in root.findall('.//image')[:-1] if img.get('id', '-1').isdigit()] # 追加した要素は除く
            new_id = max(all_ids) + 1 if all_ids else 0

            image_element.set('id', str(new_id))
            image_element.set('name', image_name)
            image_element.set('width', str(img_width))
            image_element.set('height', str(img_height))
            # 他の画像からsubsetとtask_idを取得 (存在すれば)
            first_image = root.find('.//image')
            subset_val = 'Train' # デフォルト値
            task_id_val = ''     # デフォルト値 (空にするか、必要なら設定)
            if first_image is not None:
                 subset_val = first_image.get('subset', subset_val)
                 # task_id はオプションかもしれないので存在チェック
                 if first_image.get('task_id') is not None:
                     task_id_val = first_image.get('task_id')

            image_element.set('subset', subset_val)
            if task_id_val: # task_idが存在する場合のみ設定
                image_element.set('task_id', task_id_val)
            print(f"        Added new <image> tag (id={new_id}, name={image_name}, subset={subset_val}" + (f", task_id={task_id_val}" if task_id_val else "") + ")")
        else:
            print(f"        In visualization mode, skipping image not found in XML.")
            continue # XML要素がない場合はこの画像をスキップ (視覚化モード)

    processed_images_count += 1
    print(f"  Processing corresponding <image> tag (id={image_element.get('id')}) in XML.")

    # 視覚化するかどうかのフラグ
    should_visualize_this_image = visualize_results and (visualize_limit <= 0 or visualized_count < visualize_limit)

    # 視覚化する場合、描画用の画像コピーを作成
    if should_visualize_this_image:
        vis_img = original_img.copy()
    else:
        vis_img = None # 視覚化しない場合はNone

    # --- ここからがループ内の主要処理 ---

    # --- YOLO Inference ---
    try:
        results = model(str(image_path), verbose=False) # 推論ログ抑制
        print(f"  Inference successful.")
    except Exception as e:
        print(f"  Error during YOLO inference for {image_name}: {e}")
        continue
    # --------------------

    # 既存の 'auto' ソースのアノテーションを削除 (視覚化モードでない場合のみ)
    if not visualize_results:
        existing_auto_ellipses = image_element.findall(".//ellipse[@source='auto']")
        if existing_auto_ellipses:
            print(f"  Removing {len(existing_auto_ellipses)} existing auto-generated annotations.")
            for ellipse in list(existing_auto_ellipses): # イテレート中に削除するためリストコピー
                image_element.remove(ellipse)

    # 検出結果を処理 (XMLへの追加は視覚化モードでない場合のみ)
    detections_in_image = 0
    for result in results:
        if result.obb is None or len(result.obb) == 0: # OBBがない、または空の場合スキップ
             # 初回の結果でOBBがない場合のみメッセージ表示
             if detections_in_image == 0 and results.index(result) == 0:
                 print("  No OBB detections found in the first result batch.")
             continue

        for obb in result.obb:
            # --- OBBデータを抽出 ---
            cls_id = int(obb.cls.item())
            class_name = class_names.get(cls_id, f"Unknown_{cls_id}")
            conf = obb.conf.item()
            # OBBの座標点を取得 (4点 x 2座標 = 8要素)
            # xyxyxyxy 形式: [x1, y1, x2, y2, x3, y3, x4, y4]
            # これを (-1, 2) に reshape して [[x1, y1], [x2, y2], [x3, y3], [x4, y4]] の形式にする
            pts = obb.xyxyxyxy.cpu().numpy().astype(np.float32).reshape(-1, 2)

            # --- 楕円パラメータを計算 ---
            try:
                # Center (cx, cy) - 4点の平均として計算
                center = np.mean(pts, axis=0)
                cx = center[0]
                cy = center[1]

                # 隣接点間の距離で辺の長さを計算 (cv2.minAreaRectより安定する場合がある)
                side1_len = np.linalg.norm(pts[0] - pts[1])
                side2_len = np.linalg.norm(pts[1] - pts[2])

                # 半径を計算 (0除算防止のため最小値を0.1とする)
                rx_base = max(0.1, side1_len / 2.0)
                ry_base = max(0.1, side2_len / 2.0)

                # 描画/基準角度を計算 (pts[0]からpts[1]へのベクトルを使用)
                # このベクトルが楕円の描画角度の基準となる
                vec_vis = pts[1] - pts[0]
                angle_vis_rad = math.atan2(vec_vis[1], vec_vis[0]) # Y座標が第1引数、X座標が第2引数
                angle_vis_deg = math.degrees(angle_vis_rad)

                # --- CVAT用パラメータ計算 ---
                # CVATのellipseはrxが長軸半径、ryが短軸半径、rotationが長軸の角度(X軸基準)
                # ここでは、OBBの side2 の長さに基づいて rx_cvat を、side1 に基づいて ry_cvat を初期設定
                rx_cvat = ry_base # side2/2 を rx (長軸候補) に
                ry_cvat = rx_base # side1/2 を ry (短軸候補) に
                rotation_cvat = angle_vis_deg # まず描画角度を基準とする

                # もし実際に side2 が長軸だった場合 (ry_base > rx_base)
                # rotation_cvat は side1 の方向の角度なので、長軸(side2)の角度にするために90度加算
                if ry_base > rx_base:
                    # この時点で rx_cvat と ry_cvat は正しい (ry_base が大きいので rx_cvat が長軸半径)
                    # rotation_cvat を side2 の角度に合わせる
                    rotation_cvat += 90
                # else (side1 が長軸 or 等しい場合):
                #   rx_cvat (ry_base) と ry_cvat (rx_base) を入れ替える必要がある
                #   rotation_cvat は angle_vis_deg のままで良い (side1が長軸なので)
                elif rx_base > ry_base:
                    rx_cvat, ry_cvat = ry_cvat, rx_cvat # 半径を入れ替え
                    # rotation_cvat は変更不要

                # 角度を0-360度の範囲に正規化
                rotation_cvat = rotation_cvat % 360
                # Pythonの % 演算子は負数を返すことがあるため、負なら360を加算
                if rotation_cvat < 0:
                    rotation_cvat += 360
                # --- ここまでCVAT用パラメータ計算 ---

                detections_in_image += 1 # 有効な検出としてカウント

                # --- XML要素を作成 (視覚化モードでない場合のみ) ---
                if not visualize_results:
                    ellipse = ET.SubElement(image_element, 'ellipse')
                    ellipse.set('label', class_name)
                    ellipse.set('source', 'auto') # 自動生成されたことを示す
                    ellipse.set('occluded', '0') # オクルージョンなし (デフォルト)
                    # 数値を小数点以下2桁の文字列にフォーマット
                    ellipse.set('cx', f"{cx:.2f}")
                    ellipse.set('cy', f"{cy:.2f}")
                    ellipse.set('rx', f"{rx_cvat:.2f}") # CVAT用 長軸半径
                    ellipse.set('ry', f"{ry_cvat:.2f}") # CVAT用 短軸半径
                    ellipse.set('rotation', f"{rotation_cvat:.2f}") # CVAT用 長軸の回転角度
                    ellipse.set('z_order', '0') # z_orderを追加 (CVAT標準)
                    # 必要であれば他の属性 (例: group_id, attributes) を追加
                    annotations_added_count += 1 # XMLに追加した場合のみカウント
                    print(f"    Adding ellipse to XML: label='{class_name}', cx={cx:.2f}, cy={cy:.2f}, rx={rx_cvat:.2f}, ry={ry_cvat:.2f}, rotation={rotation_cvat:.2f}")

                # --- 視覚化 (Visualisation) ---
                if should_visualize_this_image and vis_img is not None:
                    # OBBを描画 (緑色)
                    obb_pts_int = pts.astype(np.int32)
                    cv2.polylines(vis_img, [obb_pts_int], isClosed=True, color=(0, 255, 0), thickness=1)

                    # 計算された楕円を描画 (赤色) - 描画コードのパラメータを使用
                    # cv2.ellipseのaxesは (width/2, height/2) なので、rx_base, ry_base を使う
                    center_int = (int(round(cx)), int(round(cy)))
                    axes_vis = (int(round(rx_base)), int(round(ry_base))) # 描画用
                    angle_vis = angle_vis_deg # 描画用

                    cv2.ellipse(vis_img, center_int, axes_vis, angle_vis, 0, 360, (0, 0, 255), 1)

                    # ラベル描画
                    label_text = f"{class_name} {conf:.2f}"
                    # テキストをOBBの左上に配置（単純化）
                    text_x = min(obb_pts_int[:, 0])
                    text_y = min(obb_pts_int[:, 1]) - 5
                    text_y = max(10, text_y) # 画像上端にはみ出ないように
                    cv2.putText(vis_img, label_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)

                    #視覚化モードでもログは表示 (表示用パラメータとCVAT用パラメータ両方)
                    if visualize_results:
                         print(f"    Detection (for vis): label='{class_name}', conf={conf:.2f}, cx={cx:.2f}, cy={cy:.2f}, rx_vis={rx_base:.2f}, ry_vis={ry_base:.2f}, angle_vis={angle_vis_deg:.2f}")
                         print(f"                     (for XML):                         rx_cvat={rx_cvat:.2f}, ry_cvat={ry_cvat:.2f}, rotation_cvat={rotation_cvat:.2f}")


            except Exception as calc_e:
                print(f"    Error during ellipse parameter calculation or drawing for an OBB: {calc_e}")
                print(f"      OBB points: {pts}")
                # このOBBはスキップして次のOBBへ
                continue

    # 画像内の全OBB処理後
    if detections_in_image == 0 and not visualize_results: # 視覚化モードでない場合のみメッセージ表示
        print("  No valid detections were added to the XML for this image.")
    elif detections_in_image == 0 and visualize_results:
         print("  No valid detections found for visualization in this image.")

    # --- 画像表示 (matplotlibでインライン表示) ---
    if should_visualize_this_image and detections_in_image > 0 and vis_img is not None:
        print(f"  Displaying results (Image {visualized_count + 1}/{visualize_limit if visualize_limit > 0 else 'unlimited'})...")
        # OpenCV (BGR) から Matplotlib (RGB) へ変換
        vis_img_rgb = cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 8)) # 表示サイズを調整
        plt.imshow(vis_img_rgb)
        plt.title(f"OBB(Green) vs Ellipse(Red, based on OBB sides) - {image_name}")
        plt.axis('off') # 軸を非表示
        plt.show() # インライン表示
        visualized_count += 1
    elif should_visualize_this_image and detections_in_image == 0:
        # 検出がない場合は表示しないが、カウンターは進めない
        print("  Skipping visualization for this image as there were no detections.")

# --- 全画像の処理ループ終了後 ---

# --- XML書き込み (視覚化モードでない場合のみ) ---
if not visualize_results:
    # indent関数を適用して整形（Python 3.9以降が必要）
    if hasattr(ET, 'indent'):
        try:
            ET.indent(tree, space="  ", level=0) # スペース2つでインデント
            print("\nXML tree indentation applied.")
        except Exception as indent_e:
            print(f"\nWarning: Error during XML indentation: {indent_e}. Saving without pretty printing.")

    try:
        # XMLファイルを書き込む前に存在確認とバックアップ（オプション）
        output_path = Path(output_xml_path)
        if output_path.exists():
            print(f"Warning: Output file {output_path} already exists. It will be overwritten.")
            # backup_path = output_path.with_suffix(output_path.suffix + '.bak')
            # print(f"Creating backup: {backup_path}")
            # try:
            #     output_path.rename(backup_path)
            # except OSError as backup_e:
            #     print(f"Error creating backup file: {backup_e}. Overwriting existing file.")

        # tree.write() はElementTreeオブジェクト全体を書き込む
        # xml_declaration=True で <?xml version='1.0' encoding='utf-8'?> を追加
        # short_empty_elements=False で <tag/> の代わりに <tag></tag> を使う (CVAT互換性のため推奨)
        tree.write(output_xml_path, encoding='utf-8', xml_declaration=True, short_empty_elements=False)
        print(f"\nSuccessfully updated and saved XML to: {output_xml_path}")
        print(f"Processed {processed_images_count} images found (or created) in the XML.")
        print(f"Added/Updated {annotations_added_count} ellipse annotations with source='auto'.")
    except Exception as e:
        print(f"\nError writing updated XML file {output_xml_path}: {e}")
else:
    print(f"\nVisualization mode was enabled. XML file '{output_xml_path}' was NOT saved.")
    if visualize_limit > 0:
        print(f"Displayed {visualized_count} out of a maximum of {visualize_limit} images.")
    else:
        print(f"Displayed {visualized_count} images.")


print("\nScript finished.")


  Inference successful.
    Adding ellipse to XML: label='Iris', cx=515.28, cy=481.30, rx=151.38, ry=150.62, rotation=0.23
    Adding ellipse to XML: label='Pupil', cx=525.92, cy=478.19, rx=50.07, ry=49.95, rotation=270.04

Processing image: 141-20200401-15-150610_d0fb0280e261c994a47feab0975494452d606f46b508a1235ee091176d9c046c_L.jpg
  Processing corresponding <image> tag (id=568) in XML.
  Inference successful.
    Adding ellipse to XML: label='Iris', cx=456.39, cy=498.18, rx=157.74, ry=139.28, rotation=0.07
    Adding ellipse to XML: label='Pupil', cx=442.86, cy=493.18, rx=48.90, ry=36.75, rotation=0.23

Processing image: 141-20200401-15-150610_d0fb0280e261c994a47feab0975494452d606f46b508a1235ee091176d9c046c_R.jpg
  Processing corresponding <image> tag (id=569) in XML.
  Inference successful.
    Adding ellipse to XML: label='Iris', cx=512.24, cy=464.43, rx=150.62, ry=149.12, rotation=0.31
    Adding ellipse to XML: label='Pupil', cx=526.26, cy=460.34, rx=47.95, ry=47.68, rotation=27

In [33]:
import zipfile
import os

# 圧縮するXMLファイルのパス (前のセルで定義されているはず)
# xml_file_to_zip = output_xml_path # output_xml_path は前のセルで定義されていると仮定

# 出力するZIPファイルの名前を定義
# XMLファイルと同じベース名で拡張子を .zip にする
try:
    # output_xml_path が定義されているか確認
    if 'output_xml_path' not in locals() and 'output_xml_path' not in globals():
        raise NameError("変数 'output_xml_path' が定義されていません。前のセルを実行してください。")

    xml_file_to_zip = output_xml_path
    zip_file_path = os.path.splitext(xml_file_to_zip)[0] + ".zip"

    print(f"圧縮対象のXMLファイル: {xml_file_to_zip}")
    print(f"作成するZIPファイル: {zip_file_path}")

    # 書き込みモードでZipFileオブジェクトを作成
    # zipfile.ZIP_DEFLATED は圧縮を行うことを指定
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # XMLファイルをZIPアーカイブに追加
        # os.path.basename を使用して、アーカイブ内にはファイル名のみが保存されるようにする
        zipf.write(xml_file_to_zip, os.path.basename(xml_file_to_zip))

    print(f"\n'{os.path.basename(xml_file_to_zip)}' を '{os.path.basename(zip_file_path)}' に正常に圧縮しました。")
    print(f"ZIPファイルは次の場所に保存されました: {zip_file_path}")

except NameError as ne:
    print(f"エラー: {ne}")
except FileNotFoundError:
    print(f"エラー: ファイル '{xml_file_to_zip}' が見つかりませんでした。XMLファイルが正しく生成されているか確認してください。")
except Exception as e:
    print(f"ZIP圧縮中に予期せぬエラーが発生しました: {e}")


圧縮対象のXMLファイル: C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_1to160_cvat_updated.xml
作成するZIPファイル: C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_1to160_cvat_updated.zip

'1-295obb_1to160_cvat_updated.xml' を '1-295obb_1to160_cvat_updated.zip' に正常に圧縮しました。
ZIPファイルは次の場所に保存されました: C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_1to160_cvat_updated.zip
